In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True) 
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)

In [52]:

df=pd.read_csv("df_final_summarised.csv")
df.columns = df.columns.str.replace('.','_')

In [53]:
df.tail(5)[['bid_close','target1']]

bid_close  target1
15264     0.7132   0.7122
15265     0.7122   0.7128
15266     0.7128   0.7128
15267     0.7128   0.7142
15268     0.7142      NaN

In [54]:
df.loc[[15267]].bid_close

15267    0.7128
Name: bid_close, dtype: float64

In [55]:
X=df.copy()
X.drop(index=15268,inplace=True)

In [56]:
X.head()

time_local start_time_local  end_time_local  bid_open  bid_high  \
0  2010-01-05 18:00:00   1/5/2010 18:00  1/5/2010 22:00    0.9132    0.9153   
1  2010-01-05 22:00:00   1/5/2010 22:00   1/6/2010 2:00    0.9123    0.9126   
2  2010-01-06 02:00:00    1/6/2010 2:00   1/6/2010 6:00    0.9114    0.9138   
3  2010-01-06 06:00:00    1/6/2010 6:00  1/6/2010 10:00    0.9134    0.9143   
4  2010-01-06 10:00:00   1/6/2010 10:00  1/6/2010 14:00    0.9140    0.9151   

   bid_low  bid_close  PC2_AUD_H4  PC7_AUD_H4  PC14_AUD_H4  PC20_AUD_H4  \
0   0.9110     0.9124       -0.31        0.50         1.72          0.0   
1   0.9090     0.9117       -0.16       -0.03         1.64          0.0   
2   0.9102     0.9135        0.12        0.13         1.84          0.0   
3   0.9123     0.9142        0.27        0.14         1.76          0.0   
4   0.9097     0.9142        0.08       -0.01         2.13          0.0   

   PC50_AUD_H4  STDEV_AUD_H4_1p  STDEV_AUD_H4_2p  STDEV_AUD_H4_4p  \
0          0.0           0.0004           0.0012           0.0010   
1          0.0           0.0004           0.0006           0.0013   
2          0.0           0.0009           0.0007           0.0012   
3          0.0           0.0004           0.0011           0.0009   
4          0.0           0.0000           0.0003           0.0010   

   STDEV_AUD_H4_7p  STDEV_AUD_H4_10p  STDEV_AUD_H4_14p  STDEV_AUD_H4_20p  \
0           0.0020            0.0065            0.0076               0.0   
1           0.0011            0.0049            0.0073               0.0   
2           0.0011            0.0027            0.0070               0.0   
3           0.0011            0.0018            0.0065               0.0   
4           0.0011            0.0011            0.0059               0.0   

   CS_AUD_CAD_H4_7  correl_AUD_CAD_7p  CS_AUD_CAD_H4_10  correl_AUD_CAD_10p  \
0            0.706             -0.540               NaN                 NaN   
1            0.848             -0.546               NaN                 NaN   
2            0.894             -0.510               NaN                 NaN   
3            0.915             -0.466               NaN                 NaN   
4            0.932             -0.562               NaN                 NaN   

   CS_AUD_CAD_H4_20  correl_AUD_CAD_20p  CS_AUD_CHF_H4_7  correl_AUD_CHF_7p  \
0               NaN                 NaN            0.937             -0.538   
1               NaN                 NaN            0.878             -0.657   
2               NaN                 NaN            0.849             -0.627   
3               NaN                 NaN            0.787             -0.433   
4               NaN                 NaN            0.690             -0.481   

   CS_AUD_CHF_H4_10  correl_AUD_CHF_10p  CS_AUD_CHF_H4_20  correl_AUD_CHF_20p  \
0               NaN                 NaN               NaN                 NaN   
1               NaN                 NaN               NaN                 NaN   
2               NaN                 NaN               NaN                 NaN   
3               NaN                 NaN               NaN                 NaN   
4               NaN                 NaN               NaN                 NaN   

   CS_AUD_GBP_H4_7  correl_AUD_GBP_7p  CS_AUD_GBP_H4_10  correl_AUD_GBP_10p  \
0           -0.571             -0.495               NaN                 NaN   
1           -0.735              0.273               NaN                 NaN   
2           -0.727              0.409               NaN                 NaN   
3           -0.719              0.337               NaN                 NaN   
4           -0.644              0.421               NaN                 NaN   

   CS_AUD_GBP_H4_20  correl_AUD_GBP_20p  CS_AUD_JPY_H4_7  correl_AUD_JPY_7p  \
0               NaN                 NaN           -0.423             -0.628   
1               NaN                 NaN           -0.606             -0.307   
2               NaN                 NaN           -0.469         

In [7]:
X=X.drop(['Date_x','Date_y','date_local','Datetime','time_local','start_time_local','end_time_local','class_target1','class_target2','class_target3', 'target2', 'target3','PC_AUD_H4_target2','PC_AUD_H4_target1','PC_AUD_H4_target3','Date_x', 'Date_y', 'date_local'],axis=1)

#'class_target1'

In [8]:
X.fillna(0,inplace=True)

obj=X.loc[:, X.dtypes =='object']
for col in obj:
    obj[col] = pd.to_numeric(obj[col], errors='coerce')
X[obj.columns]=obj
X=X[X.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]

In [35]:
data=X.corr()

In [9]:
X.head()

bid_open  bid_high  bid_low  bid_close  PC2_AUD_H4  PC7_AUD_H4  \
0    0.9132    0.9153   0.9110     0.9124       -0.31        0.50   
1    0.9123    0.9126   0.9090     0.9117       -0.16       -0.03   
2    0.9114    0.9138   0.9102     0.9135        0.12        0.13   
3    0.9134    0.9143   0.9123     0.9142        0.27        0.14   
4    0.9140    0.9151   0.9097     0.9142        0.08       -0.01   

   PC14_AUD_H4  PC20_AUD_H4  PC50_AUD_H4  STDEV_AUD_H4_1p  STDEV_AUD_H4_2p  \
0         1.72          0.0          0.0           0.0004           0.0012   
1         1.64          0.0          0.0           0.0004           0.0006   
2         1.84          0.0          0.0           0.0009           0.0007   
3         1.76          0.0          0.0           0.0004           0.0011   
4         2.13          0.0          0.0           0.0000           0.0003   

   STDEV_AUD_H4_4p  STDEV_AUD_H4_7p  STDEV_AUD_H4_10p  STDEV_AUD_H4_14p  \
0           0.0010           0.0020            0.0065            0.0076   
1           0.0013           0.0011            0.0049            0.0073   
2           0.0012           0.0011            0.0027            0.0070   
3           0.0009           0.0011            0.0018            0.0065   
4           0.0010           0.0011            0.0011            0.0059   

   STDEV_AUD_H4_20p  CS_AUD_CAD_H4_7  correl_AUD_CAD_7p  CS_AUD_CAD_H4_10  \
0               0.0            0.706             -0.540               0.0   
1               0.0            0.848             -0.546               0.0   
2               0.0            0.894             -0.510               0.0   
3               0.0            0.915             -0.466               0.0   
4               0.0            0.932             -0.562               0.0   

   correl_AUD_CAD_10p  CS_AUD_CAD_H4_20  correl_AUD_CAD_20p  CS_AUD_CHF_H4_7  \
0                 0.0               0.0                 0.0            0.937   
1                 0.0               0.0                 0.0            0.878   
2                 0.0               0.0                 0.0            0.849   
3                 0.0               0.0                 0.0            0.787   
4                 0.0               0.0                 0.0            0.690   

   correl_AUD_CHF_7p  CS_AUD_CHF_H4_10  correl_AUD_CHF_10p  CS_AUD_CHF_H4_20  \
0             -0.538               0.0                 0.0               0.0   
1             -0.657               0.0                 0.0               0.0   
2             -0.627               0.0                 0.0               0.0   
3             -0.433               0.0                 0.0               0.0   
4             -0.481               0.0                 0.0               0.0   

   correl_AUD_CHF_20p  CS_AUD_GBP_H4_7  correl_AUD_GBP_7p  CS_AUD_GBP_H4_10  \
0                 0.0           -0.571             -0.495               0.0   
1                 0.0           -0.735              0.273               0.0   
2                 0.0           -0.727              0.409               0.0   
3                 0.0           -0.719              0.337               0.0   
4                 0.0           -0.644              0.421               0.0   

   correl_AUD_GBP_10p  CS_AUD_GBP_H4_20  correl_AUD_GBP_20p  CS_AUD_JPY_H4_7  \
0                 0.0               0.0                 0.0           -0.423   
1                 0.0               0.0                 0.0           -0.606   
2                 0.0               0.0                 0.0           -0.469   
3                 0.0               0.0                 0.0            0.014   
4                 0.0               0.0                 0.0            0.510   

   correl_AUD_JPY_7p  CS_AUD_JPY_H4_10  correl_AUD_JPY_10p  CS_AUD_JPY_H4_20  \
0             -0.628               0.0                 0.0               0.0   
1             -0.307               0.0                 0.0               0.0   
2             -0.137               0.0         

In [10]:
X['target1']=X['target1']*10000
y=X['target1']
X=X.drop(['target1'],axis=1)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.seed=0

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(9629, 713) (9629,)
(4127, 713) (4127,)


In [13]:
#clf = linear_model.Lasso(alpha=0)
clf=Ridge()
#clf=LinearRegression()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
y_train_pred=clf.predict(X_train)

In [14]:
r2_score(y_test,y_pred)

0.9995475343724106

In [15]:
r2_score(y_train, y_train_pred)

0.9996060125283903

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_train, y_train_pred) 

16.81560178727357

In [17]:
mean_absolute_error(y_test, y_pred)

17.782056168569603

In [18]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean()) 
                   
tr=rmse(y_train, y_train_pred)
te=rmse(y_test, y_pred)                 
print("rms error is train: " + str(tr)) 
print("rms error is test: " + str(te)) 

rms error is train: 24.164895527622384
rms error is test: 26.020470149253725


In [19]:
yhat=clf.predict(X_train)
SS_Residual=sum((y_train-yhat)**2)
SS_Total=sum((y_train-np.mean(y_train))**2)
r_squared=1-(float(SS_Residual))/SS_Total
adjusted_r_squared=1 - (1-r_squared)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print (r_squared, adjusted_r_squared )

0.9996060125283903 0.9995745023694158


In [20]:
yhat=clf.predict(X_test)
SS_Residual=sum((y_test-yhat)**2)
SS_Total=sum((y_test-np.mean(y_test))**2)
r_squared=1-(float(SS_Residual))/SS_Total
adjusted_r_squared=1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print (r_squared, adjusted_r_squared )

0.9995475343724106 0.9994530110813261


In [21]:
from pandas import Series
from matplotlib import pyplot
series = (y_test-y_pred)
pyplot.figure(1)
pyplot.subplot(211)
series.hist()
pyplot.subplot(212)
series.plot(kind='kde',label='residual')
pyplot.legend()

In [22]:
y_test1=pd.DataFrame(y_test)
y_pred1=pd.DataFrame(y_pred)
y_test1.columns=['y_test']
y_pred1.columns=['y_pred']

y_pred1.index=y_test1.index

df_plot=pd.concat([X_test,y_test1,y_pred1],axis=1)
df_plot.head()

bid_open  bid_high  bid_low  bid_close  PC2_AUD_H4  PC7_AUD_H4  \
14288    0.7645    0.7650   0.7610     0.7617       -0.47       -0.72   
11264    0.7653    0.7698   0.7650     0.7696        0.31        0.65   
14442    0.7367    0.7387   0.7366     0.7380        0.24       -0.61   
2817     1.0465    1.0475   1.0425     1.0439       -0.20       -0.81   
1487     0.9845    0.9891   0.9838     0.9851        0.00       -0.43   

       PC14_AUD_H4  PC20_AUD_H4  PC50_AUD_H4  STDEV_AUD_H4_1p  \
14288        -0.17        -0.42         0.98           0.0014   
11264         0.12        -0.08         0.92           0.0021   
14442        -1.06        -0.67        -0.03           0.0006   
2817          0.09         0.39         0.15           0.0013   
1487         -0.16        -1.69        -2.75           0.0005   

       STDEV_AUD_H4_2p  STDEV_AUD_H4_4p  STDEV_AUD_H4_7p  STDEV_AUD_H4_10p  \
14288           0.0015           0.0014           0.0016            0.0014   
11264           0.0017           0.0015           0.0015            0.0015   
14442           0.0007           0.0013           0.0021            0.0033   
2817            0.0011           0.0029           0.0028            0.0026   
1487            0.0005           0.0025           0.0028            0.0025   

       STDEV_AUD_H4_14p  STDEV_AUD_H4_20p  CS_AUD_CAD_H4_7  correl_AUD_CAD_7p  \
14288            0.0021            0.0018            0.879             -0.749   
11264            0.0019            0.0023           -0.735             -0.828   
14442            0.0037            0.0037            0.580             -0.935   
2817             0.0039            0.0043           -0.294             -0.489   
1487             0.0029            0.0057            0.253             -0.833   

       CS_AUD_CAD_H4_10  correl_AUD_CAD_10p  CS_AUD_CAD_H4_20  \
14288             0.903              -0.746            -0.337   
11264             0.336              -0.851             0.154   
14442             0.495              -0.913             0.586   
2817              0.191              -0.461             0.932   
1487              0.945              -0.681             0.908   

       correl_AUD_CAD_20p  CS_AUD_CHF_H4_7  correl_AUD_CHF_7p  \
14288              -0.769           -0.212              0.726   
11264               0.538           -0.361             -0.834   
14442              -0.912            0.672             -0.887   
2817               -0.590           -0.726             -0.803   
1487               -0.919            0.469             -0.306   

       CS_AUD_CHF_H4_10  correl_AUD_CHF_10p  CS_AUD_CHF_H4_20  \
14288            -0.627               0.656            -0.292   
11264            -0.571              -0.805             0.446   
14442            -0.517              -0.752             0.762   
2817             -0.720              -0.612             0.813   
1487              0.829              -0.289             0.830   

       correl_AUD_CHF_20p  CS_AUD_GBP_H4_7  correl_AUD_GBP_7p  \
14288               0.048            0.504              0.123   
11264              -0.235           -0.228             -0.118   
14442              -0.755            0.390              0.891   
2817                0.269           -0.362              0.703   
1487               -0.815           -0.382              0.152   

       CS_AUD_GBP_H4_10  correl_AUD_GBP_10p  CS_AUD_GBP_H4_20  \
14288             0.302               0.056            -0.730   
11264            -0.286              -0.339            -0.404   
14442            -0.397               0.846             0.790   
2817             -0.870               0.574             0.496   
1487              0.151               0.266            -0.255   

       correl_AUD_GBP_20p  CS_AUD_JPY_H4_7  correl_AUD_JPY_7p  \
14288               0.292            0.856              0.798   
11264               0.622            0.742             -0.711   
14442               0.712            0.370             -0.845   
2817  

In [23]:
df_plot['diff']=abs(df_plot['y_test']-df_plot['y_pred'])
df_plot['diff_bid_close']=abs(df_plot['y_test']-df_plot['bid_close']*10000)
sorted_df=df_plot.sort_values(by='diff',ascending=False)
sorted_df.head(10)

bid_open  bid_high  bid_low  bid_close  PC2_AUD_H4  PC7_AUD_H4  \
3274     0.9992    0.9997   0.9938     0.9968       -0.92       -0.01   
5994     0.9501    0.9551   0.9498     0.9549        0.52        0.89   
1358     0.9847    0.9901   0.9841     0.9883       -0.02        0.38   
2947     1.0026    1.0040   0.9874     0.9887       -1.74       -3.78   
666      0.8132    0.8193   0.8128     0.8138        0.56       -1.94   
2946     1.0066    1.0072   0.9990     1.0026       -0.22       -2.26   
647      0.8125    0.8241   0.8086     0.8208       -0.70       -3.40   
146      0.8794    0.8809   0.8780     0.8784       -0.31       -1.28   
2959     0.9783    0.9837   0.9748     0.9786        0.01        0.29   
11649    0.7760    0.7771   0.7717     0.7731       -0.10        0.08   

       PC14_AUD_H4  PC20_AUD_H4  PC50_AUD_H4  STDEV_AUD_H4_1p  \
3274          1.19         2.71        -0.26           0.0013   
5994         -0.61        -0.84         0.70           0.0024   
1358          0.06        -0.49         0.85           0.0019   
2947         -3.26        -3.52        -4.21           0.0070   
666          -2.23         0.16        -8.61           0.0002   
2946         -1.78        -2.55        -2.99           0.0018   
647          -5.35        -5.79        -8.70           0.0041   
146          -0.18        -0.55        -2.84           0.0007   
2959         -2.74        -4.60        -4.13           0.0002   
11649         0.78         0.70         1.95           0.0013   

       STDEV_AUD_H4_2p  STDEV_AUD_H4_4p  STDEV_AUD_H4_7p  STDEV_AUD_H4_10p  \
3274            0.0039           0.0031           0.0029            0.0052   
5994            0.0023           0.0027           0.0025            0.0026   
1358            0.0018           0.0023           0.0023            0.0023   
2947            0.0075           0.0106           0.0128            0.0128   
666             0.0020           0.0038           0.0078            0.0074   
2946            0.0015           0.0084           0.0098            0.0095   
647             0.0058           0.0062           0.0111            0.0114   
146             0.0011           0.0015           0.0033            0.0033   
2959            0.0002           0.0013           0.0020            0.0026   
11649           0.0011           0.0019           0.0019            0.0022   

       STDEV_AUD_H4_14p  STDEV_AUD_H4_20p  CS_AUD_CAD_H4_7  correl_AUD_CAD_7p  \
3274             0.0060            0.0116            0.188             -0.463   
5994             0.0044            0.0054            0.648             -0.851   
1358             0.0022            0.0027            0.000             -0.665   
2947             0.0116            0.0100            0.969             -0.982   
666              0.0077            0.0074            0.997             -0.973   
2946             0.0085            0.0075            0.947             -0.977   
647              0.0153            0.0200            0.912             -0.936   
146              0.0031            0.0037           -0.017             -0.901   
2959             0.0096            0.0198            0.993             -0.415   
11649            0.0026            0.0025            0.577             -0.917   

       CS_AUD_CAD_H4_10  correl_AUD_CAD_10p  CS_AUD_CAD_H4_20  \
3274              0.932              -0.831             0.975   
5994              0.887              -0.709             0.267   
1358              0.725              -0.587             0.087   
2947              0.947              -0.987             0.761   
666               0.927              -0.976             0.973   
2946              0.834              -0.980             0.453   
647               0.077              -0.947             0.561   
146               0.560              -0.905            -0.743   
2959              0.997              -0.587             0.989   
11649             0.329              -0.936             0.306   

       correl_AUD_CA

In [24]:
df_plot['PCT_DIFF_Pred']= (df_plot['y_pred']  - (df_plot['bid_close'] * 10000 )) / (df_plot['bid_close'] * 10000  )
df_plot['PCT_DIFF_Target']= (df_plot['y_test'] - (df_plot['bid_close'] * 10000)  ) / (df_plot['bid_close'] * 10000  ) 

In [25]:
df_plot.loc[df_plot['PCT_DIFF_Target'] < - 0.0005, 'class_target1'] = -1
df_plot.loc[df_plot['PCT_DIFF_Target'] > 0.0005, 'class_target1'] = 1
df_plot.loc[(df_plot['PCT_DIFF_Target'] >= - 0.0005 ) &  (df_plot['PCT_DIFF_Target'] <= 0.0005 ), 'class_target1'] = 0 

In [26]:
df_plot.loc[df_plot['PCT_DIFF_Pred'] < - 0.0005, 'class_target1_pred'] = -1
df_plot.loc[df_plot['PCT_DIFF_Pred'] > 0.0005, 'class_target1_pred'] = 1
df_plot.loc[(df_plot['PCT_DIFF_Pred'] >= - 0.0005) &  (df_plot['PCT_DIFF_Pred'] <= 0.0005 ) , 'class_target1_pred'] = 0

In [30]:
from sklearn.metrics import classification_report,accuracy_score

In [28]:
print(classification_report( df_plot.class_target1, df_plot.class_target1_pred  )) 

             precision    recall  f1-score   support

       -1.0       0.38      0.22      0.28      1591
        0.0       0.23      0.57      0.33       925
        1.0       0.40      0.24      0.30      1611

avg / total       0.35      0.31      0.30      4127



In [31]:
print(accuracy_score(df_plot.class_target1, df_plot.class_target1_pred )) 

0.305064211291495


In [32]:
pd.crosstab(df_plot.class_target1, df_plot.class_target1_pred) 

class_target1_pred  -1.0   0.0   1.0
class_target1                       
-1.0                 355   856   380
 0.0                 211   524   190
 1.0                 373   858   380

In [50]:
sorted_df.shape

(4127, 717)

In [36]:
li=sorted_df[sorted_df.columns[1:]].corr()['y_test'][:-1].to_frame().reset_index()
li.dropna(axis=0,inplace=True) 

In [37]:
sorted_Df2=li.sort_values(by='y_test',ascending=False)

In [38]:
cols_new=li.loc[(li['y_test'] >= 0.3) | (li['y_test'] <= -0.3)]['index'].tolist() 

In [40]:
cols_new.remove('y_test')
cols_new.remove('y_pred')

In [58]:
cols_new.append('target1')
new_try_df=X[cols_new]
new_try_df.head()


bid_high  bid_low  bid_close  VOL_ATR_AUD5  VOL_ATR_AUD6  VOL_ATR_AUD7  \
0    0.9153   0.9110     0.9124      0.003669      0.003669      0.003669   
1    0.9126   0.9090     0.9117      0.003655      0.003657      0.003659   
2    0.9138   0.9102     0.9135      0.003644      0.003648      0.003650   
3    0.9143   0.9123     0.9142      0.003315      0.003373      0.003415   
4    0.9151   0.9097     0.9142      0.003732      0.003711      0.003698   

   VOL_ATR_AUD8  VOL_ATR_AUD9  VOL_ATR_AUD10  VOL_ATR_AUD11  VOL_ATR_AUD12  \
0      0.003669      0.003669       0.003669       0.003669       0.003669   
1      0.003660      0.003661       0.003662       0.003662       0.003663   
2      0.003653      0.003654       0.003656       0.003657       0.003658   
3      0.003446      0.003470       0.003490       0.003506       0.003520   
4      0.003690      0.003685       0.003681       0.003678       0.003676   

   VOL_ATR_AUD13  VOL_ATR_AUD14  VOL_ATR_AUD15  VOL_ATR_AUD16  VOL_ATR_AUD17  \
0       0.003669       0.003669       0.003669       0.003669       0.003669   
1       0.003663       0.003664       0.003664       0.003664       0.003665   
2       0.003659       0.003659       0.003660       0.003660       0.003661   
3       0.003531       0.003541       0.003549       0.003557       0.003563   
4       0.003675       0.003674       0.003673       0.003672       0.003671   

   VOL_ATR_AUD18  VOL_ATR_AUD19  VOL_ATR_AUD20  VOL_BOLLINGER_BANDS_AUD2  \
0       0.003669       0.003669       0.003669                  0.913040   
1       0.003665       0.003665       0.003665                  0.913040   
2       0.003661       0.003662       0.003662                  0.915146   
3       0.003569       0.003574       0.003579                  0.914840   
4       0.003671       0.003670       0.003670                  0.914200   

   VOL_BOLLINGER_BANDS_AUD3  VOL_BOLLINGER_BANDS_AUD4  \
0                  0.914348                  0.915180   
1                  0.914348                  0.915180   
2                  0.914348                  0.915180   
3                  0.915713                  0.915180   
4                  0.914775                  0.915761   

   VOL_BOLLINGER_BANDS_AUD5  VOL_BOLLINGER_BANDS_AUD6  \
0                  0.915432                  0.916397   
1                  0.915432                  0.916397   
2                  0.915432                  0.916397   
3                  0.915432                  0.916397   
4                  0.915432                  0.916397   

   VOL_BOLLINGER_BANDS_AUD7  VOL_BOLLINGER_BANDS_AUD8  \
0                  0.918723                  0.920517   
1                  0.918723                  0.920517   
2                  0.918723                  0.920517   
3                  0.918723                  0.920517   
4                  0.918723                  0.920517   

   VOL_BOLLINGER_BANDS_AUD9  VOL_BOLLINGER_BANDS_AUD10  \
0                  0.923681                    0.92425   
1                  0.923681                    0.92425   
2                  0.923681                    0.92425   
3                  0.923681                    0.92425   
4                  0.923681                    0.92425   

   VOL_BOLLINGER_BANDS_AUD11  VOL_BOLLINGER_BANDS_AUD12  \
0                   0.924188                   0.924169   
1                   0.924188                   0.924169   
2                   0.924188                   0.924169   
3                   0.924188                   0.924169   
4                   0.924188                   0.924169   

   VOL_BOLLINGER_BANDS_AUD13  VOL_BOLLINGER_BANDS_AUD14  \
0                   0.923993                   0.923748   
1                   0.923993                   0.923748   
2                   0.923993                   0.923748   
3                   0.923993                   0.923748   
4                   0.923993                   0.923748   

   VOL_BOLLINGER_BANDS_AUD15  VOL_BOLLINGER_B

In [60]:
y=new_try_df['target1']*10000
new_try_df.drop(['target1'],axis=1,inplace=True)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(new_try_df, y, test_size=0.3, random_state=1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(10687, 178) (10687,)
(4581, 178) (4581,)


In [49]:
new_try_df_1.shape

(15233, 178)

In [62]:
#clf = linear_model.Lasso(alpha=0)
clf=Ridge()
#clf=LinearRegression()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
y_train_pred=clf.predict(X_train)

In [63]:
r2_score(y_test,y_pred)

0.9994681995755538

In [64]:
r2_score(y_train, y_train_pred)

0.9994994161069674

In [65]:
mean_absolute_error(y_train, y_train_pred) 

19.55942064122426

In [66]:
mean_absolute_error(y_test, y_pred)

19.09692795092587

In [67]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean()) 
                   
tr=rmse(y_train, y_train_pred)
te=rmse(y_test, y_pred)                 
print("rms error is train: " + str(tr)) 
print("rms error is test: " + str(te)) 

rms error is train: 27.387069067288127
rms error is test: 28.001559847256456


In [68]:
yhat=clf.predict(X_train)
SS_Residual=sum((y_train-yhat)**2)
SS_Total=sum((y_train-np.mean(y_train))**2)
r_squared=1-(float(SS_Residual))/SS_Total
adjusted_r_squared=1 - (1-r_squared)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print (r_squared, adjusted_r_squared )

0.9994994161069674 0.9994909364787832


In [69]:
yhat=clf.predict(X_test)
SS_Residual=sum((y_test-yhat)**2)
SS_Total=sum((y_test-np.mean(y_test))**2)
r_squared=1-(float(SS_Residual))/SS_Total
adjusted_r_squared=1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print (r_squared, adjusted_r_squared )

0.9994681995755538 0.999446695605642


In [70]:
from pandas import Series
from matplotlib import pyplot
series = (y_test-y_pred)
pyplot.figure(1)
pyplot.subplot(211)
series.hist()
pyplot.subplot(212)
series.plot(kind='kde',label='residual')
pyplot.legend()

In [71]:
y_test1=pd.DataFrame(y_test)
y_pred1=pd.DataFrame(y_pred)
y_test1.columns=['y_test']
y_pred1.columns=['y_pred']

y_pred1.index=y_test1.index

df_plot=pd.concat([X_test,y_test1,y_pred1],axis=1)
df_plot.head()

bid_high  bid_low  bid_close  VOL_ATR_AUD5  VOL_ATR_AUD6  VOL_ATR_AUD7  \
9966     0.7155   0.7138     0.7145      0.002819      0.002887      0.002930   
12971    0.7920   0.7897     0.7914      0.002185      0.002187      0.002194   
2411     1.0713   1.0672     1.0696      0.005153      0.005235      0.005291   
4029     1.0345   1.0317     1.0327      0.002940      0.002988      0.003017   
13441    0.7597   0.7578     0.7585      0.002442      0.002488      0.002525   

       VOL_ATR_AUD8  VOL_ATR_AUD9  VOL_ATR_AUD10  VOL_ATR_AUD11  \
9966       0.002958      0.002978       0.002992       0.003002   
12971      0.002205      0.002218       0.002233       0.002247   
2411       0.005330      0.005358       0.005378       0.005393   
4029       0.003037      0.003053       0.003066       0.003079   
13441      0.002553      0.002574       0.002589       0.002599   

       VOL_ATR_AUD12  VOL_ATR_AUD13  VOL_ATR_AUD14  VOL_ATR_AUD15  \
9966        0.003010       0.003017       0.003022       0.003027   
12971       0.002263       0.002278       0.002293       0.002308   
2411        0.005405       0.005415       0.005424       0.005432   
4029        0.003091       0.003103       0.003115       0.003127   
13441       0.002605       0.002609       0.002610       0.002609   

       VOL_ATR_AUD16  VOL_ATR_AUD17  VOL_ATR_AUD18  VOL_ATR_AUD19  \
9966        0.003032       0.003037       0.003041       0.003045   
12971       0.002322       0.002337       0.002350       0.002364   
2411        0.005440       0.005448       0.005457       0.005465   
4029        0.003139       0.003150       0.003162       0.003173   
13441       0.002606       0.002603       0.002598       0.002594   

       VOL_ATR_AUD20  VOL_BOLLINGER_BANDS_AUD2  VOL_BOLLINGER_BANDS_AUD3  \
9966        0.003050                  0.714774                  0.714850   
12971       0.002377                  0.792223                  0.792874   
2411        0.005473                  1.069791                  1.070598   
4029        0.003184                  1.035188                  1.034765   
13441       0.002588                  0.760031                  0.759758   

       VOL_BOLLINGER_BANDS_AUD4  VOL_BOLLINGER_BANDS_AUD5  \
9966                   0.715236                  0.716257   
12971                  0.792407                  0.792251   
2411                   1.071963                  1.071420   
4029                   1.034495                  1.034333   
13441                  0.759595                  0.761040   

       VOL_BOLLINGER_BANDS_AUD6  VOL_BOLLINGER_BANDS_AUD7  \
9966                   0.719061                  0.721261   
12971                  0.792075                  0.791934   
2411                   1.071262                  1.072009   
4029                   1.034238                  1.035183   
13441                  0.761129                  0.762782   

       VOL_BOLLINGER_BANDS_AUD8  VOL_BOLLINGER_BANDS_AUD9  \
9966                   0.722307                  0.722265   
12971                  0.791764                  0.791633   
2411                   1.072427                  1.073320   
4029                   1.035159                  1.039887   
13441                  0.764525                  0.765660   

       VOL_BOLLINGER_BANDS_AUD10  VOL_BOLLINGER_BANDS_AUD11  \
9966                    0.722301                   0.721985   
12971                   0.791610                   0.791508   
2411                    1.073682                   1.073224   
4029                    1.042594                   1.043877   
13441                   0.766170                   0.765856   

       VOL_BOLLINGER_BANDS_AUD12  VOL_BOLLINGER_BANDS_AUD13  \
9966                    0.721863                   0.722004   
12971                   0.791454                   0.791372   
2411                    1.072901                   1.073853   
4029                    1.044694                   1.045836   
13441                   0.765594   

In [72]:
df_plot['diff']=abs(df_plot['y_test']-df_plot['y_pred'])
df_plot['diff_bid_close']=abs(df_plot['y_test']-df_plot['bid_close']*10000)
sorted_df=df_plot.sort_values(by='diff',ascending=False)
sorted_df.head(10)

bid_high  bid_low  bid_close  VOL_ATR_AUD5  VOL_ATR_AUD6  VOL_ATR_AUD7  \
8446    0.8323   0.8323     0.8323      0.002452      0.002608      0.002733   
2745    1.0200   1.0130     1.0141      0.012866      0.012614      0.012318   
5994    0.9551   0.9498     0.9549      0.004630      0.004703      0.004776   
8447    0.8678   0.8678     0.8678      0.009062      0.008090      0.007414   
2740    1.0318   1.0176     1.0188      0.009229      0.009016      0.008851   
2947    1.0040   0.9874     0.9887      0.010334      0.009874      0.009513   
3038    0.9946   0.9878     0.9936      0.007143      0.007041      0.006970   
2943    1.0231   1.0182     1.0220      0.006108      0.006193      0.006254   
636     0.8586   0.8540     0.8561      0.007909      0.007720      0.007533   
5919    0.9670   0.9630     0.9635      0.004361      0.004478      0.004571   

      VOL_ATR_AUD8  VOL_ATR_AUD9  VOL_ATR_AUD10  VOL_ATR_AUD11  VOL_ATR_AUD12  \
8446      0.002837      0.002925       0.003000       0.003065       0.003122   
2745      0.012020      0.011734       0.011465       0.011213       0.010977   
5994      0.004847      0.004917       0.004984       0.005047       0.005107   
8447      0.006920      0.006544       0.006250       0.006014       0.005820   
2740      0.008711      0.008587       0.008473       0.008365       0.008262   
2947      0.009223      0.008987       0.008791       0.008626       0.008487   
3038      0.006922      0.006891       0.006873       0.006865       0.006864   
2943      0.006301      0.006340       0.006373       0.006402       0.006428   
636       0.007362      0.007209       0.007076       0.006960       0.006859   
5919      0.004643      0.004699       0.004743       0.004777       0.004804   

      VOL_ATR_AUD13  VOL_ATR_AUD14  VOL_ATR_AUD15  VOL_ATR_AUD16  \
8446       0.003172       0.003215       0.003254       0.003289   
2745       0.010756       0.010550       0.010356       0.010173   
5994       0.005163       0.005216       0.005264       0.005309   
8447       0.005658       0.005521       0.005404       0.005302   
2740       0.008163       0.008068       0.007976       0.007887   
2947       0.008366       0.008262       0.008170       0.008087   
3038       0.006868       0.006876       0.006887       0.006901   
2943       0.006450       0.006470       0.006487       0.006502   
636        0.006772       0.006696       0.006629       0.006569   
5919       0.004826       0.004843       0.004856       0.004867   

      VOL_ATR_AUD17  VOL_ATR_AUD18  VOL_ATR_AUD19  VOL_ATR_AUD20  \
8446       0.003319       0.003347       0.003372       0.003394   
2745       0.010001       0.009839       0.009686       0.009540   
5994       0.005350       0.005388       0.005422       0.005453   
8447       0.005212       0.005133       0.005063       0.004999   
2740       0.007801       0.007719       0.007639       0.007563   
2947       0.008013       0.007945       0.007883       0.007826   
3038       0.006916       0.006932       0.006949       0.006967   
2943       0.006513       0.006523       0.006530       0.006535   
636        0.006515       0.006467       0.006422       0.006381   
5919       0.004876       0.004882       0.004887       0.004890   

      VOL_BOLLINGER_BANDS_AUD2  VOL_BOLLINGER_BANDS_AUD3  \
8446                  0.832300                  0.832957   
2745                  1.015440                  1.015578   
5994                  0.959288                  0.957268   
8447                  0.900255                  0.885125   
2740                  1.040431                  1.045780   
2947                  1.015308                  1.017649   
3038                  0.998537                  1.000472   
2943                  1.023097                  1.027505   
636                   0.857197                  0.864090   
5919                  0.964840                  0.964572   

      VOL_BOLLINGER_BANDS_AUD4  VOL_BOLLINGER_BANDS_AUD5  \
8446          